In [1]:
import pandas as pd
# from sklearn.preprocessing import StandardScaler
# import numpy as np
# from sklearn.metrics import silhouette_score
# from sklearn.cluster import KMeans
# from yellowbrick.cluster import KElbowVisualizer
# import matplotlib.pyplot as plt
# import seaborn as sns
# from mpl_toolkits.mplot3d import Axes3D

In [2]:
distribution_centers = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/distribution_centers.csv')
event = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/events.csv')
inventory_items = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/inventory_items.csv')
order_items = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/order_items.csv')
orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/orders.csv')
products = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/products.csv')
users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/users.csv')

# RFM-D(Diversity) 데이터 전처리

In [55]:
#필요한 데이터 전처리
order_items_extracted = order_items.query('status != "Returned" and status != "Cancelled"')
order_items_extracted = order_items_extracted[['user_id', 'order_id', 'created_at', 'status', 'product_id', 'sale_price']].sort_values(by = 'user_id')

# 년-월-일 형식의 문자열로 변환
order_items_extracted['created_at'] = pd.to_datetime(order_items_extracted['created_at'], format='ISO8601')
order_items_extracted['created_at'] = pd.to_datetime(order_items_extracted['created_at'].dt.strftime('%Y-%m-%d'))
order_items_extracted = order_items_extracted.rename(columns = {'created_at' : 'date'}).query('date < "2023-12-01"')

In [68]:
#recency
recency_data = order_items_extracted.groupby('user_id')['date'].max().reset_index()
recency_data['Recency'] = (order_items_extracted['date'].max()-recency_data['date']).dt.days
recency_data

#frequency
frequency_data = order_items_extracted.groupby('user_id')['order_id'].nunique().reset_index()
frequency_data.rename(columns={'order_id': 'Frequency'}, inplace=True)

#monetary
monetary_data = order_items_extracted.groupby('user_id')['sale_price'].sum().reset_index()
monetary_data.rename(columns={'sale_price': 'Monetary'}, inplace=True)

#diversity
diversity_data = order_items_extracted.groupby('user_id')['product_id'].nunique().reset_index()
diversity_data.rename(columns={'product_id': 'Diversity'}, inplace=True)

rfm_data = recency_data.merge(frequency_data, on='user_id').merge(monetary_data, on='user_id').merge(diversity_data, on='user_id')

In [69]:
rfm_data

,user_id,date,Recency,Frequency,Monetary,Diversity
0,1,2022-07-20,498,1,159.990000,3
1,2,2022-02-20,648,1,22.000000,1
2,3,2023-08-08,114,4,402.200001,5
3,4,2022-10-18,408,1,148.000000,1
4,6,2022-04-11,598,1,144.820004,2
...,...,...,...,...,...,...
58459,99989,2023-09-01,90,1,15.120000,2
58460,99992,2022-08-09,478,1,55.950001,3
58461,99996,2023-03-15,260,2,123.870001,4
58462,99998,2023-07-22,131,3,116.990000,3


# **추가 전처리**
## R : percentiles = [97.5, 86.5, 66, 34, 13.5, 2.5] : 6점 점수화
## F : 구매수 + 접속수
## M : percentiles = [97.5, 86.5, 66, 34, 13.5, 2.5] 6점 점수화
## D : 품목수 + 카테고리수

## 접속수 구하기 : 5회 이상이면 2점, 4회 이하는 1점


In [58]:
event_filtered = event[event['user_id'].notna()].sort_values(by = ['user_id', 'created_at'])


In [59]:
event_filtered['created_at'] = pd.to_datetime(event_filtered['created_at'], format='ISO8601')
event_filtered['created_at'] = pd.to_datetime(event_filtered['created_at'].dt.strftime('%Y-%m-%d'))
event_filtered = event_filtered.rename(columns = {'created_at' : 'date'}).query('date < "2023-12-01"')

In [60]:
event_filtered_curated = event_filtered.query('sequence_number == 1').groupby('user_id')['sequence_number'].count().reset_index()
event_filtered_curated = event_filtered_curated.rename(columns= {"sequence_number": "connect_counts"})

In [70]:
rfm_data

,user_id,date,Recency,Frequency,Monetary,Diversity
0,1,2022-07-20,498,1,159.990000,3
1,2,2022-02-20,648,1,22.000000,1
2,3,2023-08-08,114,4,402.200001,5
3,4,2022-10-18,408,1,148.000000,1
4,6,2022-04-11,598,1,144.820004,2
...,...,...,...,...,...,...
58459,99989,2023-09-01,90,1,15.120000,2
58460,99992,2022-08-09,478,1,55.950001,3
58461,99996,2023-03-15,260,2,123.870001,4
58462,99998,2023-07-22,131,3,116.990000,3


In [71]:
rfm_temp = rfm_data.merge(event_filtered_curated, on = 'user_id').rename(columns = {'connect_counts' : 'access_counts'})
rfm_data = rfm_temp[['user_id', 'Recency','Frequency', 'Monetary', 'Diversity', 'access_counts']].rename(columns = {'Diversity': 'prouduct_id_counts'})
rfm_data = rfm_data.drop(columns=['prodouct_id_counts'])

,user_id,Recency,Frequency,Monetary,prouduct_id_counts,access_counts
0,1,498,1,159.990000,3,3
1,2,648,1,22.000000,1,1
2,3,114,4,402.200001,5,5
3,4,408,1,148.000000,1,1
4,6,598,1,144.820004,2,2
...,...,...,...,...,...,...
58459,99989,90,1,15.120000,2,2
58460,99992,478,1,55.950001,3,3
58461,99996,260,2,123.870001,4,4
58462,99998,131,3,116.990000,3,3


In [ ]:
#최초 주문일 및 최초 접속일

event_first_ones = event.groupby('user_id')['created_at'].min().reset_index().rename(columns = {'created_at' : 'first_access'})
event_last_ones = event.groupby('user_id')['created_at'].max().reset_index().rename(columns = {'created_at': 'last_access'})
event_first_and_last = event_first_ones.merge(event_last_ones, on = 'user_id')
#!!!!!!! event_first_and_last

In [79]:
event_first_and_last['user_id'] = event_first_and_last['user_id'].astype('int64')

In [ ]:
order_items_first_ones = order_items.groupby('user_id')['created_at'].min().reset_index().rename(columns = {'created_at' : 'fisrt_order'})
order_items_last_ones = order_items.groupby('user_id')['created_at'].max().reset_index().rename(columns = {'created_at' : 'last_order'})

order_items_first_and_last = order_items_first_ones.merge(order_items_last_ones, on = 'user_id')

####order_items_first_and_last

In [92]:
#product칼럼
order_items_info = order_items[['user_id', 'product_id']].merge(products[['id', 'category', 'brand', 'department', 'distribution_center_id']], left_on = 'product_id', right_on = 'id')
order_items_info_by_user = order_items_info.groupby('user_id').agg({'product_id':'nunique',
                                         'category':'nunique',
                                         'brand':'nunique',
                                         'department':'nunique',
                                         'distribution_center_id':'nunique'}).reset_index()
order_items_info_by_user = (order_items_info_by_user.rename(columns = {'product_id':'product_counts', 'category': 'category_counts',
                                                                       'brand': 'brand_counts', 'department':'department_counts', 'distribution_center_id' : 'distribution_center_counts'}))

80044

In [103]:
#users 칼럼
user_extracted = users[['id', 'age', 'gender','country', 'state', 'traffic_source', 'created_at']].rename(columns = {'id': 'user_id', 'created_at':'id_created_date'})
###user_extracted

In [ ]:
user_browser_count = event.groupby('user_id')['browser'].nunique().reset_index().rename(columns = {'browser' : 'browser_counts'})
####user_browser_count

여기서부터 사용!


In [106]:
merged = (event_first_and_last.merge(order_items_first_and_last, on='user_id').
          merge(order_items_info_by_user, on = 'user_id').
          merge(user_extracted, on = 'user_id').
          merge(user_browser_count, on ='user_id'))

In [ ]:
rfm_final = rfm_data.merge(merged, on = 'user_id')


In [117]:
rfm_final.columns

Index(['user_id', 'Recency', 'Frequency', 'Monetary', 'access_counts',
       'first_access', 'last_access', 'fisrt_order', 'last_order',
       'product_counts', 'category_counts', 'brand_counts',
       'department_counts', 'distribution_center_counts', 'age', 'gender',
       'country', 'state', 'traffic_source', 'id_created_date',
       'browser_counts'],
      dtype='object')

In [114]:
rfm_final.to_csv('rfm_final.csv', index=False)

In [115]:
rfm_final.to_parquet('rfm_final.parquet', index=False)